In [1]:
import os
import mne
import pandas as pd
import numpy as np
import janitor

data_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp'
final_dataset_path = '/home/owner/Documents/DEV/BrainLabyrinth/data/final_dataset'
for user_id in os.listdir(data_path):
    person_dir = os.path.join(data_path, user_id)
    for file in os.listdir(person_dir):
        file_path = os.path.join(person_dir, file)

        if file.endswith(".vhdr"):
            # Read the EEG data
            raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)

            # Find the corresponding .vmrk file
            vmrk_file = file.replace('.vhdr', '.vmrk')
            vmrk_file_path = os.path.join(person_dir, vmrk_file)

            if os.path.exists(vmrk_file_path):
                # Read the annotations (markers) from the .vmrk file
                annotations = mne.annotations.read_annotations(vmrk_file_path)

                # Add the annotations to the raw object
                # raw.set_annotations(annotations)
                # print("Markers added from the .vmrk file")

            # Convert raw data to DataFrame
            time_series = raw.to_data_frame()

            # Extract markers (annotations)
            # print(f"{annotations=}")
            marker_times = annotations.onset  # In seconds
            marker_labels = annotations.description  # The marker labels

            # Create a DataFrame for the markers
            markers_df = pd.DataFrame({
                'event_id': np.arange(len(marker_labels)), 
                'start': marker_times - 3.0,
                'end': marker_times + 1.0,
                'marker': marker_labels
            })

            # Display the first few rows of both DataFrames
            # Merge markers with EEG data
            time_series['time'] = time_series['time'].round(3)  # Round times to 3 decimal places for matching

            merged_df = janitor.conditional_join(
                markers_df, 
                time_series,
                ('start', 'time', '<='),
                ('end', 'time', '>='),
                how='left',
                df_columns=['event_id', 'marker']
            )

            merged_path = os.path.join(final_dataset_path, f"{user_id}.parquet")
            merged_df.to_parquet(merged_path)


Finding 'sfreq' from header file: /home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp/15/Alim_Maze_15_2024-06-05_19-12-59.vhdr


/tmp/ipykernel_2997214/3426309007.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)


Finding 'sfreq' from header file: /home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp/25/Alim_Maze_25_2024-06-14_19-37-38.vhdr


/tmp/ipykernel_2997214/3426309007.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)


Finding 'sfreq' from header file: /home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp/30/Alim_Maze_29_2024-06-19_20-01-02.vhdr


...

/tmp/ipykernel_2997214/3426309007.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)


Finding 'sfreq' from header file: /home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp/46/Alim_Maze_46_2024-06-28_18-42-28.vhdr


/tmp/ipykernel_2997214/3426309007.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)


Finding 'sfreq' from header file: /home/owner/Documents/DEV/BrainLabyrinth/data/Ymaze_exp/08/Alim_Maze_08_2024-05-30_13-47-25.vhdr


/tmp/ipykernel_2997214/3426309007.py:16: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_path, preload=True, ignore_marker_types=True, verbose=False)
